In [1]:
import os, scanpy as sc, warnings
from geneformer import TranscriptomeTokenizer, EmbExtractor
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

In [2]:
dataset = "eye"
DATA_DIR = "/home/wangh256/Islander_dev/data/%s/" % dataset
adata = sc.read_h5ad(rf"{DATA_DIR}/emb.h5ad")
adata.X = adata.layers["raw_counts"]
adata

AnnData object with n_obs × n_vars = 48981 × 29673
    obs: 'biosample_id', 'donor_id', 'cell_type_ontology_term_id', 'organism_ontology_term_id', 'disease_ontology_term_id', 'tissue_ontology_term_id', 'assay_ontology_term_id', 'cell_type__custom', 'development_stage_ontology_term_id', 'sex_ontology_term_id', 'suspension_type', 'is_primary_data', 'age', 'self_reported_ethnicity_ontology_term_id', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'n_counts', 'n_genes', '_scvi_batch', '_scvi_labels', 'conditions_combined'
    var: 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype', 'n_cells'
    uns: '_scvi_manager_uuid', '_scvi_uuid', 'cell_type_colors', 'log1p', 'neighbors', 'pca', 'schema_version', 'title', 'tsne', 'umap'
    obsm: 'Geneformer', 'Geneformer_UMAP', 'Harmony', 'Harmony_UMAP', 'Islander', 'Islander_SCL', 'Islander_SCL_UMAP', 'Islander_TRI', 'Islander_TRI_UMAP', 'Islander_UMAP', 'Provid

In [3]:
adata.var.head()

,feature_is_filtered,feature_name,feature_reference,feature_biotype,n_cells
gene_ids,,,,,
ENSG00000243485,False,MIR1302-2HG,NCBITaxon:9606,gene,113
ENSG00000237613,False,FAM138A,NCBITaxon:9606,gene,17
ENSG00000238009,False,RP11-34P13.7,NCBITaxon:9606,gene,254
ENSG00000241860,False,RP11-34P13.13,NCBITaxon:9606,gene,646
ENSG00000241599,False,RP11-34P13.9,NCBITaxon:9606,gene,13


In [4]:
adata.var['ensembl_id'] = adata.var.index.tolist()

In [5]:
os.makedirs(rf"{DATA_DIR}/Geneformer", exist_ok=True)
os.makedirs(rf"{DATA_DIR}/GeneformerRaw", exist_ok=True)
adata.write_h5ad(rf"{DATA_DIR}/GeneformerRaw/local.h5ad", compression="gzip")

tk = TranscriptomeTokenizer({}, nproc=16)
tk.tokenize_data(rf"{DATA_DIR}/GeneformerRaw", 
                 rf"{DATA_DIR}/Geneformer", 
                 dataset, 
                 file_format="h5ad")

Tokenizing /home/wangh256/Islander_dev/data/eye/GeneformerRaw/local.h5ad
/home/wangh256/Islander_dev/data/eye/GeneformerRaw/local.h5ad has no column attribute 'filter_pass'; tokenizing all cells.
Creating dataset.


Map (num_proc=16):   0%|          | 0/48981 [00:00<?, ? examples/s]

Map (num_proc=16):   0%|          | 0/48981 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/48981 [00:00<?, ? examples/s]

In [6]:
embex = EmbExtractor(model_type="Pretrained",
                     num_classes=0,
                     max_ncells=None,
                     forward_batch_size=128,
                     nproc=16)

embs = embex.extract_embs("/home/wangh256/GeneDataEngine_dev/Geneformer-main/geneformer-12L-30M",
                          rf"{DATA_DIR}/Geneformer/{dataset}.dataset",
                          rf"{DATA_DIR}/",
                          "Geneformer")

  0%|          | 0/383 [00:00<?, ?it/s]

In [7]:
embs

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,-0.002395,-0.013618,-0.009898,0.003715,0.027983,0.044488,-0.019582,-0.009240,-0.034118,0.034324,...,0.013263,0.030504,0.018553,0.031204,0.014214,-0.021961,-0.029018,-0.011996,0.002177,-0.025248
1,0.007458,-0.033755,0.009877,0.003998,0.020734,0.058412,0.005678,-0.015380,-0.017890,0.046616,...,0.037718,0.019069,-0.005409,0.045154,0.023234,-0.031534,-0.020503,-0.043775,0.013293,-0.039457
2,-0.005619,-0.027241,-0.005561,-0.003682,0.008809,0.041009,0.022319,0.007139,-0.032794,0.090452,...,-0.007362,0.028629,0.006897,0.019762,0.045937,-0.044144,-0.022708,-0.043145,0.029624,-0.026715
3,0.000959,-0.027369,-0.008465,0.001370,-0.003953,0.037561,-0.024571,-0.015594,-0.039167,0.046891,...,0.015203,-0.003330,0.029155,0.016561,0.021866,-0.002586,-0.003752,-0.045425,0.005103,-0.016290
4,-0.011275,-0.019907,-0.019691,0.003722,0.011628,0.033808,-0.027955,0.018790,-0.029486,0.048524,...,0.023187,0.003533,0.019235,0.018353,0.032400,-0.021172,-0.030966,-0.018062,0.012432,-0.030218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48976,0.025174,-0.001727,0.026214,0.000973,-0.003128,0.021305,-0.009780,0.018237,0.001709,-0.021251,...,0.028916,0.018966,0.000022,0.022634,-0.005928,-0.014672,-0.011444,0.014179,-0.002151,-0.016365
48977,0.029809,-0.030063,0.022909,-0.000005,0.005120,0.033246,-0.011090,0.025449,-0.024464,-0.005825,...,0.015554,0.032800,0.021001,0.011944,-0.004278,-0.015013,-0.016721,0.010995,-0.001858,-0.030193
48978,0.028608,0.002319,0.024588,-0.000784,-0.004131,0.021955,0.008164,0.027745,0.007088,-0.001086,...,0.026314,0.020785,0.002720,0.016647,0.002657,-0.010621,-0.020980,0.012103,0.006307,-0.033755
48979,0.030072,-0.023417,0.014408,0.001479,-0.019943,0.023076,-0.008544,0.023940,-0.014923,-0.008359,...,0.007809,0.022821,0.019562,0.010346,-0.005769,-0.009558,-0.017058,0.028475,-0.006424,-0.008761


In [8]:
adata.obsm["Geneformer"] = embs.values